# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("WeatherPy.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Country,Date,Cloudiness,Wind Speed
0,ushuaia,-54.8000,-68.3000,37.06,81,AR,1650856675,0,5.75
1,oberstdorf,47.4000,10.2833,42.35,94,DE,1650856676,100,4.00
2,guerrero negro,27.9769,-114.0611,65.28,60,MX,1650856676,0,18.77
3,perg,48.2500,14.6333,49.42,92,AT,1650856676,100,11.79
4,grand-santi,4.2500,-54.3833,72.84,99,GF,1650856677,97,2.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]
# humidity

In [4]:
fig = gmaps.figure(center = (20,0), zoom_level=2.2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=350,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clouds = weather_df['Cloudiness']
temp = weather_df['Max Temp']
wind = weather_df['Wind Speed']

fit_weather = weather_df.loc[(clouds == 0)]
fit_weather = fit_weather.loc[(temp > 70) & (temp < 80)]
fit_weather = fit_weather.loc[(wind < 10)]
fit_weather

,City,Lat,Lng,Max Temp,Humidity,Country,Date,Cloudiness,Wind Speed
8,laguna,38.4210,-121.4238,73.54,42,US,1650856678,0,8.05
13,jamestown,42.0970,-79.2353,71.24,55,US,1650856481,0,6.91
131,cabo san lucas,22.8909,-109.9124,72.70,65,MX,1650856422,0,3.00
211,caravelas,-17.7125,-39.2481,74.88,93,BR,1650856745,0,3.15
231,moctezuma,29.8000,-109.6667,71.33,18,MX,1650856752,0,5.10
237,beckley,37.7782,-81.1882,70.21,43,US,1650856754,0,6.91
310,praia,14.9215,-23.5087,73.94,46,CV,1650856780,0,9.22
324,port hueneme,34.1478,-119.1951,71.73,58,US,1650856785,0,8.05
342,oildale,35.4197,-119.0195,73.42,33,US,1650856791,0,6.91
386,lawrenceburg,38.0373,-84.8966,73.29,70,US,1650856807,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = fit_weather
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    results = response['results']
    
    try:
        print(f"Hotel Name {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results for Index 8: laguna.
Hotel Name Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
Retrieving Results for Index 13: jamestown.
Hotel Name Hampton Inn & Suites Jamestown.
Retrieving Results for Index 131: cabo san lucas.
Hotel Name Hotel Tesoro Los Cabos.
Retrieving Results for Index 211: caravelas.
Hotel Name Pousada dos Navegantes.
Retrieving Results for Index 231: moctezuma.
Hotel Name Hotel Lobohombo.
Retrieving Results for Index 237: beckley.
Hotel Name Country Inn & Suites by Radisson, Beckley, WV.
Retrieving Results for Index 310: praia.
Hotel Name LT Aparthotel.
Retrieving Results for Index 324: port hueneme.
Hotel Name Hampton Inn Channel Islands Harbor.
Retrieving Results for Index 342: oildale.
Hotel Name DoubleTree by Hilton Hotel Bakersfield.
Retrieving Results for Index 386: lawrenceburg.
Hotel Name Best Western Lawrenceburg Inn.
Retrieving Results for Index 401: santarem.
Hotel Name Hotel Imperador.
Retrieving Results for Index 447: 

In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Country,Date,Cloudiness,Wind Speed,Hotel Name
8,laguna,38.4210,-121.4238,73.54,42,US,1650856678,0,8.05,Holiday Inn Express & Suites Elk Grove Central...
13,jamestown,42.0970,-79.2353,71.24,55,US,1650856481,0,6.91,Hampton Inn & Suites Jamestown
131,cabo san lucas,22.8909,-109.9124,72.70,65,MX,1650856422,0,3.00,Hotel Tesoro Los Cabos
211,caravelas,-17.7125,-39.2481,74.88,93,BR,1650856745,0,3.15,Pousada dos Navegantes
231,moctezuma,29.8000,-109.6667,71.33,18,MX,1650856752,0,5.10,Hotel Lobohombo
237,beckley,37.7782,-81.1882,70.21,43,US,1650856754,0,6.91,"Country Inn & Suites by Radisson, Beckley, WV"
310,praia,14.9215,-23.5087,73.94,46,CV,1650856780,0,9.22,LT Aparthotel
324,port hueneme,34.1478,-119.1951,71.73,58,US,1650856785,0,8.05,Hampton Inn Channel Islands Harbor
342,oildale,35.4197,-119.0195,73.42,33,US,1650856791,0,6.91,DoubleTree by Hilton Hotel Bakersfield
386,lawrenceburg,38.0373,-84.8966,73.29,70,US,1650856807,0,6.91,Best Western Lawrenceburg Inn


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))